<img src="https://github.com/Fortuz/edu_Adaptive/blob/main/practices/assets/logo.png?raw=1" width="800">

Made by **Balázs Nagy** and **Márk Domokos**

[<img src="https://github.com/Fortuz/edu_Adaptive/blob/main/practices/assets/open_button.png?raw=1">](https://colab.research.google.com/github/Fortuz/edu_Adaptive/blob/main/practices/L07%20Neural%20Networks%20Train_solved.ipynb)

# Labor 07 - Training of a neural Network

### Handwritten numbers III.
In this exercise we will train our neural network to recognise handwritten digits from 0 to 9.

### 1: Imports and Load Data
Import the package you want to use and read the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

### 2: Data load

The data will be loaded from a publicly available file. An alternative solution would be to upload the data file directly to the google colab file system.

In [ ]:
!wget https://github.com/Fortuz/edu_Adaptive/raw/main/practices/assets/Lab07/Lab7data.mat
!wget https://github.com/Fortuz/edu_Adaptive/raw/main/practices/assets/Lab07/Lab7weights.mat
!wget https://github.com/Fortuz/edu_Adaptive/raw/main/practices/assets/Lab07/nnC_history_800.txt
!wget https://github.com/Fortuz/edu_Adaptive/raw/main/practices/assets/Lab07/w1_final.txt
!wget https://github.com/Fortuz/edu_Adaptive/raw/main/practices/assets/Lab07/w2_final.txt

Load in the data! Use the Pandas package to do this and then convert it into a numpy array.

In [ ]:
data = loadmat("Lab7data.mat")
X = data["X"]
y = data ["y"]
del data
m = X.shape[0]
print('Shape of X and y in order:')
print(X.shape)
print(y.shape)

data = loadmat("Lab7weights.mat")
w1 = np.array(data["Theta1"])
w2 = np.array(data["Theta2"])
print('Shape of w1 and w2 in order:')
print(w1.shape)
print(w2.shape)
del data

Consideration in connection with the data:
- The samples are sorted into a 5000x400 matrix, which means that we have 5000 20x20 pixel images. The images are stored expanded in the matrix.
- As an output, we expect a number, which are stored in a 5000x1 vector.
- The weight matrices required for the network to work are 25x401 and 10x26 respectively. This means that our initial input parameter space of 400 is augmented with the BIAS and the hidden layer contains 25 neurons.
- The BIAS is also appended to the hidden layer and thus creates the connection between the second weight vector of 26 hidden neurons and the 10 possible outputs.

In terms of the operation of the neural network, we will consider the probability of an element belonging to one of the 10 classes, of which we will choose the largest (MaxPooling) to determine the final number.

### 3: Visualization

Visualize the date for a better understanding.

In [ ]:
print("Printing some random data ...")
fig, ax = plt.subplots(10,10, figsize =(8,8))
for i in range(10):
    for j in range(10):
        ax[i,j].imshow(X[np.random.randint(0,m+1),:].reshape(20,20, order = "F"), cmap="hot")
        ax[i,j].axis("off")
plt.show()

### 4. Mathematical background

For a better understanding, consider the simplified example below.

<img src="https://github.com/Fortuz/edu_Adaptive/blob/main/practices/assets/Lab07/Pics/L07_Network.png?raw=1" width="550">

The outlined neural network consists of 3 layers.
- Input layer containing 2 input neurons plus the added BIAS.
- Hidden layer containing 3 neurons plus the assigned BIAS.
- Output layer containing 3 neurons, the maximum of which is selected to get the actual output.

In terms of the role of neurons, the treatment of BIAS neurons is slightly different. Since BIAS is not a variable, but a constant value of 1, so that the associated weight can be an independent parameter. Accordingly, it only has a role in weight injection and can be assigned to each layer as required.

In terms of indexes introduce the following: <br>
$a^{(j)}_{i}$ - activation of the $i^{th}$ neuron in the $j^{th}$ layer <br>
$s^{(j)}_{i}$ - summed output of the $i^{th}$ neuron in the $j^{th}$ layer <br>
$w^{(j)}_{lk}$ - weight between the $l^{th}$ neuron in the $j^{th}$ layer and the $k^{th}$ neuron in the $j+1^{th}$ layer  <br>
$x^{(m)}_{n}$ - the $n^{th}$ feauture in the $m^{th}$ input. ($x^{(1)}_0 = 1)$ is the BIAS.<br>
$\hat y$ - the output

The forward step can therefore be structured as follows: <br>
The BIAS is added to the $x^{(1)}$ vector and multiplied by the first weight matrix.

$ \underset{1\times 3}{\mathrm{x^{(1)}}} \times \underset{3\times 3}{\mathrm{w^{(1)}}} = \underset{1\times 3}{\mathrm{s^{(2)}}} $

We perform the activation in the neurons of the hidden layer. We use the sigmoid function as activation function.

$ \underset{\color{red}{1\times 3}}{\mathrm{a^{(2)}}} = f(\underset{1\times 3}{\mathrm{s^{(2)}}}) = sigmoid(\underset{1\times 3}{\mathrm{s^{(2)}}}) $

We assign the BIAS to the hidden layer after the activation, but before the weight is applied!

$ \underset{\color{red}{1\times 4}}{\mathrm{a^{(2)}}} \times \underset{4\times 3}{\mathrm{w^{(2)}}} = \underset{1\times 3}{\mathrm{s^{(3)}}}  $

After activation the value of the neurons in the output layer is obtained.

$ \underset{1\times 3}{\mathrm{a^{(3)}}} = f(\underset{1\times 3}{\mathrm{s^{(3)}}}) = sigmoid(\underset{1\times 3}{\mathrm{s^{(3)}}}) $

The output layer contains the predictions.

$ \underset{1\times 3}{\mathrm{\hat{y}}} = \underset{1\times 3}{\mathrm{a^{(3)}}} $

In a classification problem, the final prediction is obtained by selecting the maximum, i.e. the most likely group.

#### Cost function

We use the well-established MSE (Mean Square Error) method as a cost function. Let's see how the error calculation is for an output neuron. Ignoring the BIASs, let us focus only on the connections shown in dark color in the figure. <br>

$ C = \sum\{\frac{1}{2}(y-\color{red}{\hat y})^2 \} $

$ C = \sum\{\frac{1}{2}(y-{\color{red}{a^{(3)}}})^2 \} $

$ C = \sum\{\frac{1}{2}(y-f({\color{red}{s^{(3)}}}))^2\} $

$ C = \sum\{\frac{1}{2}(y-f({\color{red}{a^{(2)}}}w^{(2)}))^2\} $

$ C = \sum\{\frac{1}{2}(y-f(f({\color{red}{s^{(2)}}})w^{(2)}))^2\}$

$ C = \sum\{\frac{1}{2}(y-f(f(xw^{(1)})w^{(2)}))^2\} $

The error calculation can therefore be reduced to a function of the input variables and the weights in the mesh. The formula can be applied to any number of layers.

#### Back Propagation

The error back propagation is used to calculate the extent to which each neuron is responsible for the error, starting from the output layer and working backwards. In practice, we need to perform half of the steps used to calculate the cost function backwards, which means we need the derivative of the activation function. <br>

##### Sigmoid function and its derivative

$g(z) = \frac{1}{1+e^{-z}}$ <br>
$
\begin{split}
g'(z) =
& = \frac{d}{dz}\frac{1}{1+e^{-z}} \\
& = \frac{1}{(1+e^{-z})^2}(e^{(-z)}) \\
& = \frac{1}{1+e^{-z}}(1-\frac{1}{(1+e^{-z})}) \\
& = g(z)(1-g(z))
\end{split}
$

<img src="https://github.com/Fortuz/edu_Adaptive/blob/main/practices/assets/Lab07/Pics/L07_SigmoidDeriv.png?raw=1" width="400">

Back propagation of the error is used to derive the error rate in the neurons of the hidden layer by parsimoniously deriving the cost function according to the elements of the corresponding weight matrix.

$ \begin{split}
\frac{\partial C}{\partial w^{(2)}} = \frac{\partial \sum \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}} = \sum (\frac{\partial \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}})
\end{split}$

For the sake of clarity, let's derive the derivation for one element. <br>

$
\begin{split}
\frac{\partial \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}}
& = (y-\hat{y})(-\frac{\hat{y}}{\partial w^{(2)}}) \\
& = -(y-\hat{y}) \cdot \frac{\partial \hat{y}}{\partial s^{(3)}}\cdot \frac{\partial 	s^{(3)}}{\partial w^{(2)}}\\
& = \color{red}{-(y-\hat{y}) \cdot f'(s^{(3)})}\cdot
\frac{\partial a^{(2)}w^{(2)}}{\partial w^{(2)}}\\
& = {\color{red} {\delta^{(3)}}\cdot a^{(2)}}
\end{split}
$<br>

Introduce $ \delta^{(j)}_{i} $ as the error term assigned to neuron i of layer j. <br>
Extended in matrix form, after checking the dimensions, we obtain the following relation: <br>
$ (a^{(2)})^T \delta^{(3)} $


#### Training

During the teaching phase, we modify the weights using the Gradient Descent method as previously described. To modify the weights, we define a learning rate $(\mu)$ and optionally specify a penalty term. <br>

$ w^{(1)} = w^{(1)} - \mu \frac{\partial C}{w^{(1)}}+ regularization $

$ w^{(2)} = w^{(2)} - \mu \frac{\partial C}{w^{(2)}}+ regularization $

#### Algorithm steps

Let's summarise the steps to implement the algorithm

1, $\hspace{2mm} xw^{(1)} = s^{(2)} $

2, $\hspace{14mm} f(s^{(2)}) = a^{(2)}$

3, $\hspace{32mm} a^{(2)} w^{(2)} = s^{(3)} $

4, $\hspace{48mm} f(s^{(3)}) = \hat{y} $

5, $\hspace{2mm} C = \sum\{\frac{1}{2}(y-\hat y)^2 \} $

6, $\hspace{2mm} -(y-\hat{y}) \cdot f'(s^{(3)}) = \delta^{(3)} $

7, $\hspace{28mm} (a^{(2)})^T \delta^{(3)} =  \frac{\partial C}{\partial w^{(2)}} $

8, $\hspace{39mm} \delta ^{(3)} \cdot (w^{(2)})^T \cdot f'(s^{(2)})= \delta ^{(2)} $

9, $\hspace{78mm} x^T \delta^{(2)} =  \frac{\partial C}{\partial w^{(1)}} $

10, $\hspace{2mm} w^{(1)} = w^{(1)} - \mu \frac{\partial C}{w^{(1)}}+ regularization $

$\hspace{8mm}w^{(2)} = w^{(2)} - \mu \frac{\partial C}{w^{(2)}}+ regularization $

### 5: Parameter preparation

In [ ]:
Lambda = 1                                              # Level of penalty
input_layer_size = 400                                  # Input layer size
hidden_layer_size = 25                                  # Hidden layer size
num_labels = 10                                         # Number of Labels
nn_params = np.append(w1.flatten(), w2.flatten())       # Flatten and concatanete w1 and w2

Tracing the architecture is more transparent if the neuron numbers of each layer are stored without the BIAS members. And the weights are stored in an expanded variable. The matrix sizes can be reset based on the neuron numbers of each layer. We will also need the learning rate as a preset parameter.

### 6: Cost function

Before implementing the cost function, we need an activation function and its derivative. Again, we choose the sigmoid function as the activation function.

In [ ]:
def sigmoid(z):
    ################### CODE HERE ########################
    # Sigmoid function implemnetation


    ######################################################
    return g

def sigmoidGradient(z):
    ################### CODE HERE ########################
    # Implement the derivative of the sigmoid function as well


    ######################################################
    return g_deriv

The cost function will receive as parameters the weight parameters, the layer sizes of the mesh, as well as the samples, the labels associated with the samples and the learning rate. Since the weight parameters are passed to the function in a variable, we need to return the corresponding matrix dimensions for the weight matrices within the function.

##### One Hot Encoding

To adapt the neural network to the output, we will need to do some more tweaking. The labels of the samples are given by a number, but we are working on a 10-group classification problem. Accordingly, we will have 10 output neurons, which will give us the probability that a given sample belongs to a given class. We will therefore determine the predicted class based on the maximum likelihood. And for this transformation we will use the One Hot Encoding method (only one 1). The essence of this method is to transform each label into a vector with the same number of elements as the number of classes and only one 1 in the column of the corresponding class.

Our original labels are the numbers 0 to 9 (left), and the corresponding transformed labels are the vectors of our transformed labels (right). Note that putting the vectors of the transformed labels together is very similar to a unit matrix.

<img src="https://github.com/Fortuz/edu_Adaptive/blob/main/practices/assets/Lab07/Pics/L07_OneHot.png?raw=1" width="300">

In [ ]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
    # convert nn_params back to w1 and w2 matrices
    w1 = nn_params[:((input_layer_size + 1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size + 1)
    w2 = nn_params[((input_layer_size + 1) * hidden_layer_size):].reshape(num_labels, hidden_layer_size + 1)

    m = X.shape[0]
    C = 0
    w1_grad = np.zeros((w1.shape))
    w2_grad = np.zeros((w2.shape))

    ################### CODE HERE ########################
    # Forward Step implementation
    # Add BIAS to every layer






    ######################################################

    ################### CODE HERE ########################
    # ONE HOT ENCODING (y to Y)






    ######################################################

    ################### CODE HERE ########################
    # PenaltyPenalized cost function




    ######################################################

    ################### CODE HERE ########################
    # Back Propagation









    ######################################################
    grad = np.append(w1_grad.flatten(), w2_grad.flatten())

    return C, grad

Let's check if the implemented function works as intended.

In [ ]:
Lambda = 0
C, grad = nnCostFunction(nn_params,input_layer_size,hidden_layer_size,num_labels,X,y,Lambda)
print("Cost at debugging parameters: %.6f \nFor Lambda = 0 it should be:  0.287629" % C)
Lambda = 3
C, grad = nnCostFunction(nn_params,input_layer_size,hidden_layer_size,num_labels,X,y,Lambda)
print("Cost at debugging parameters: %.6f \nFor Lambda = 3 it should be:  0.576051" % C)

### 7: Initial weights

The initial weights are initialised with a uniform distribution over a defined interval. The weights thus start from a nonzero initial value, which allows convergence.

In [ ]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = 0.12
    W = np.random.rand(L_out,L_in+1)*(2*epsilon_init)-epsilon_init
    return W

In [ ]:
initial_w1 = randInitializeWeights(input_layer_size,hidden_layer_size)
initial_w2 = randInitializeWeights(hidden_layer_size,num_labels)
initial_nn_params = np.append(initial_w1.flatten(),initial_w2.flatten())

### 8: Gradient Descent

To train the neural network implement the gradient descent method.

In [ ]:
def gradientDescentnn(X, y, initial_nn_params, lr_rate, num_iters, Lambda, input_layer_size, hidden_layer_size,
                      num_labels):

    w1 = initial_nn_params[:((input_layer_size + 1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size + 1)
    w2 = initial_nn_params[((input_layer_size + 1) * hidden_layer_size):].reshape(num_labels, hidden_layer_size + 1)

    m = len(y)
    C_history = []

    for i in range(num_iters):
        if (i%20==0):
            print('Iteration:', i+1)
        elif (i==num_iters-1):
            print('Done!')

        nn_params = np.append(w1.flatten(), w2.flatten())
        C, grad = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)

        w1_grad = grad[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
        w2_grad = grad[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

        w1 = w1 - (lr_rate * w1_grad)
        w2 = w2 - (lr_rate * w2_grad)
        C_history.append(C)

    nn_paramsFinal = np.append(w1.flatten(), w2.flatten())
    return nn_paramsFinal, C_history

In [ ]:
# Training parameters
lr_rate = 0.9
num_iter = 400
Lambda = 0.1

nnw, nnC_history = gradientDescentnn(X,y,initial_nn_params,lr_rate,num_iter,Lambda,input_layer_size,hidden_layer_size,num_labels)
w1 = nnw[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
w2 = nnw[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

You can experiment with the parameters of the run (Lambda, learning rate, number of iterations).

To test the success of a given learning, implement a predictor function that performs the Forward Step and returns the result to its original form.

In [ ]:
def predict(w1,w2,X):
    m = X.shape[0]

    ################### CODE HERE ########################
    # Implement the Forward step







    ######################################################

    return p

Due to the small number of samples, the accuracy of the network is tested on the training data, i.e. how many of the total number of samples were correctly hit. The data are used as a basis to give the accuracy of the network as a percentage.

In [ ]:
def accuracy(pred,y):
    acc = (np.mean(pred[:,np.newaxis]==y))*100
    return acc

In [ ]:
pred = predict(w1,w2,X)
acc = accuracy(pred,y)

print('Training Set Accuracy after %.0f iteration (currently trained weights): %.2f %%' % (num_iter,acc))

w1_r = np.loadtxt("w1_final.txt")
w2_r = np.loadtxt("w2_final.txt")

pred_800 = predict(w1_r,w2_r,X)
acc_800 = accuracy(pred_800,y)
print('Training Set Accuracy after 800 iteration (loaded weights): %.2f %%' % acc_800)

### 9: Run training

We examine the learning process by plotting the cost function per interaction.

In [ ]:
plt.plot(nnC_history)
plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Cost function over the iterations (Current params)')
plt.show()

nnC_history_load=np.loadtxt("nnC_history_800.txt")
plt.plot(nnC_history_load)
plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Cost function over the iterations (Loaded params)')
plt.show()

## Implementation with high level packages

With Keras (or Tensorflow) packages, the procedures described in detail above can be implemented as simpler and more compact code. The functions are optimized. Let's see how the example above can be implemented more concisely.

In [ ]:
# imports for array-handling and plotting
import numpy as np
import matplotlib.pyplot as plt

# keras imports for the dataset and building our neural network
from keras.models import Sequential
from keras.layers.core import Dense, Activation

Create a model. Define layers, neuron numbers and activation functions.

In [ ]:
# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(400, input_shape=(400,)))
model.add(Activation('sigmoid'))

model.add(Dense(25))
model.add(Activation('sigmoid'))

model.add(Dense(10))
model.add(Activation('softmax'))

Define cost function

In [ ]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Starting from the original data, the One Hot recoding is also required here.

In [ ]:
Y = np.zeros((m, num_labels))
I = np.eye(num_labels)

for i in range(1, m+1):
    Y[i-1, :] = I[y[i-1]-1, :]

Train the neural network and extract metrics.

In [ ]:
# training the model and saving metrics in history
history = model.fit(X, Y, epochs=20, verbose = 2)

Checking the execution of a training using the extracted metrics.

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'])

<div style="text-align: right">This lab exercise uses elements from Andrew Ng's Machine Learning course.</div>